In [1]:
import pandas as pd

In [2]:
data=[]
with open('log-1st/FIX.4.2-CSCII2-FIXTEST_Messages_20230927-0.log', 'r') as file:
    for line in file:
        fields=line.rstrip().split(chr(1))
        record={}
        for item in fields:
            if item:
                key, value=item.split('=')
                record[key]=value
        data.append(record)


In [3]:
orders_send=[item for item in data if item.get('35')=='D']
orders_recv=[item for item in data if item.get('39')=='0']
cancel_send=[item for item in data if item.get('35')=='F']
cancel_recv=[item for item in data if item.get('39')=='4']

In [4]:
len(orders_send), len(orders_recv), len(cancel_send), len(cancel_recv)

(30, 14, 30, 14)

In [5]:
orders_send_dt=[{'id': item['11'], 'dt':key[:-12]} for item in orders_send for key in item if key.startswith('2023')]
orders_recv_dt=[{'id': item['11'], 'dt':key[:-12]} for item in orders_recv for key in item if key.startswith('2023')]
cancel_send_dt=[{'id': item['11'], 'dt':key[:-12]} for item in cancel_send for key in item if key.startswith('2023')]
cancel_recv_dt=[{'id': item['11'], 'dt':key[:-12]} for item in cancel_recv for key in item if key.startswith('2023')]

In [6]:
df_order_send=pd.DataFrame(orders_send_dt)
df_order_recv=pd.DataFrame(orders_recv_dt)
df_cancel_send=pd.DataFrame(cancel_send_dt)
df_cancel_recv=pd.DataFrame(cancel_recv_dt)

In [7]:
df_order=pd.merge(df_order_send, df_order_recv, on='id', how='inner', suffixes=('_send', '_recv'))
df_cancel=pd.merge(df_cancel_send, df_cancel_recv, on='id', how='inner',suffixes=('_send', '_recv'))

In [8]:
df_order.dt_send=pd.to_datetime(df_order.dt_send)
df_order.dt_recv=pd.to_datetime(df_order.dt_recv)
df_cancel.dt_send=pd.to_datetime(df_cancel.dt_send)
df_cancel.dt_recv=pd.to_datetime(df_cancel.dt_recv)

In [9]:
df_order['latency']=(df_order.dt_recv-df_order.dt_send).dt.total_seconds()*1000
df_cancel['latency']=(df_cancel.dt_recv-df_cancel.dt_send).dt.total_seconds()*1000

In [17]:
df_order.sort_values(by=['dt_send',]).to_csv('order-router.csv', index=False, sep='\t')

In [18]:
df_cancel.sort_values(by=['dt_send',]).to_csv('cancel-router.csv', index=False, sep='\t')